In [223]:
import numpy as np
from collections import defaultdict
from torch.utils import data

import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
# Generate Dataset
np.random.seed(42)

In [49]:
def generate_dataset(num_sequences=2**8):
    sequences = []
    for _ in range(num_sequences):
        token_length = np.random.randint(1, 12)
        sequence = f'{"a"*token_length}{"b"*token_length}EOS'
        sequences.append(sequence)
        
    return sequences

In [50]:
def word_encoding(sequences):
    
    # Get 1D list of all words in all sequences
    flatten = lambda l: [item for sublist in l for item in sublist]
    all_words = flatten(sequences)
    
    # Create dictionary mapping word to word frequency across all sequences
    word_to_count = defaultdict(int)
    for word in all_words:
        word_to_count[word] += 1
    word_to_count = sorted(list(word_to_count.items()), key=lambda l: -l[1]) # sorting according to frequency
    
    # List of unique words
    dictionary = [item[0] for item in word_to_count]
    dictionary.append('UNK')
    
    # Calculate lengths
    num_sequences = len(sequences)
    vocab_size = len(dictionary)
    
    # Make word to index and index to word mappings
    word_to_idx = defaultdict(lambda: vocab_size-1)
    idx_to_word = defaultdict(lambda: 'UNK')
    for idx, word in enumerate(dictionary):
        word_to_idx[word] = idx
        idx_to_word[idx] = word
    
    return word_to_idx, idx_to_word, vocab_size

In [64]:
def one_hot_encode(idx, vocab_size):
    """
    One-hot encodes a single word given its index and the size of the vocabulary.
    
    Args:
     `idx`: the index of the given word
     `vocab_size`: the size of the vocabulary
    
    Returns a 1-D numpy array of length `vocab_size`.
    """
    # Initialize the encoded array
    one_hot = np.zeros(vocab_size)
    
    # Set the appropriate element to one
    one_hot[idx] = 1.0

    return one_hot


def one_hot_encode_sequence(sequence, vocab_size):
    """
    One-hot encodes a sequence of words given a fixed vocabulary size.
    
    Args:
     `sentence`: a list of words to encode
     `vocab_size`: the size of the vocabulary
     
    Returns a 3-D numpy array of shape (num words, vocab size, 1).
    """
    # Encode each word in the sentence
    encoding = np.array([one_hot_encode(word_to_idx[word], vocab_size) for word in sequence])

    # Reshape encoding s.t. it has shape (num words, vocab size, 1)
    encoding = encoding.reshape(encoding.shape[0], encoding.shape[1], 1)
    
    return encoding

In [51]:
class Dataset(data.Dataset):
    def __init__(self, inputs, targets):
        self.X = inputs
        self.y = targets

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [101]:
def prepare_data(sequences, train_size=0.8, test_size=0.1, val_size=0.1):
    
    # Split data
    num_train = int(train_size*len(sequences))
    num_test = int(test_size*len(sequences))
    num_val = int(val_size*len(sequences))
#     print(f'{num_train}, {num_test}, {num_val}')
    
    train_seq = sequences[:num_train]
    test_seq = sequences[num_train:num_train+num_test]
    val_seq = sequences[-num_val:]
#     print(f'{len(train_seq)}, {len(test_seq)}, {len(val_seq)}')
    
    # prepare input & target sequences
    def prepare_sequences(sequences):
        inputs = []
        targets = []
        
        for sequence in sequences:
            inputs.append(sequence[:-1])
            targets.append(sequence[1:])
        
        return inputs, targets
    
    train_inputs, train_targets = prepare_sequences(train_seq)
    test_inputs, test_targets = prepare_sequences(test_seq)
    val_inputs, val_targets = prepare_sequences(val_seq)
#     print(f'{len(train_inputs)}, {len(test_inputs)}, {len(val_inputs)}')
    
    # create datasets
    train_set = Dataset(train_inputs, train_targets)
    test_set = Dataset(test_inputs, test_targets)
    val_set = Dataset(val_inputs, val_targets)
    
    return train_set, test_set, val_set

In [53]:
# RNN from scratch

In [243]:
def init_orthogonal_weights(dim1, dim2):
    
    # initialize
    weights = np.random.randn(dim1, dim2)
#     print(f'inital random: {weights}')
    if dim1 < dim2:
        weights = weights.T
        
    # QR factorization (Q = orthogonal)
    q, r = np.linalg.qr(weights)
#     print(f'q: {q}')
    
    # Make Q uniform according to https://arxiv.org/pdf/math-ph/0609050.pdf
    d = np.diag(r, 0)
    ph = np.sign(d)
    q *= ph
#     print(f'q final: {q}')

    if dim1 < dim2:
        q = q.T
        
    return q # q is orthogonal

In [55]:
def init_rnn(hidden_size, vocab_size):
    '''
    Initializes RNN
    
    Args:
        hidden_size --> hidden state dimensions
        vocab_size --> input vector dimensions

    Returns:
        U --> Weight matrix applied to input, passed to hidden state
        V --> Weight matrix from previous hidden state passed to hidden state
        W --> Weight matrix applied to output from hidden state to give final output

        bias_hidden = bias applied in hidden state
        bias_output = bias applied to output
    '''
    
    U = init_orthogonal_weights(hidden_size, vocab_size)
    V = init_orthogonal_weights(hidden_size, hidden_size)
    W = init_orthogonal_weights(vocab_size, hidden_size)
    
    bias_hidden = init_orthogonal_weights(hidden_size, hidden_size)
    bias_output = init_orthogonal_weights(vocab_size, vocab_size)
    
    return (U, V, W, bias_hidden, bias_output)

In [56]:
# Activation Functions

In [117]:
def sigmoid(x, derivative=False):
    """
    Computes sigmoid of array x
    
    Args:
        x --> input array
        derivative --> when set to True will return derivative instead of forward pass
    """
    
    x_safe = x + 1e-12
    f = 1 / (1 + np.exp(-x_safe))
    
    if derivative: 
        return f * (1 - f)
    else: 
        return f

In [118]:
def tanh(x, derivative=False):
    """
    Computes tanh of array x
    
    Args:
        x --> input array
        derivative --> when set to True will return derivative instead of forward pass
    """
    
    x_safe = x + 1e-12
    f = (np.exp(x_safe)-np.exp(-x_safe))/(np.exp(x_safe)+np.exp(-x_safe))
    
    if derivative: 
        return f * (1 - f)
    else: 
        return f

In [59]:
def softmax(x):
    """
    Computes softmax of array x
    
    Args:
        x --> input array
    """
    return np.exp(x+1e-12) / np.sum(np.exp(x+1e-12))

In [60]:
# Forward Pass

In [200]:
def forward_pass(inputs, hidden_state, parameters):
    
    U, V, W, bias_hidden, bias_output = parameters
    outputs, hidden_states = [], [hidden_state]
#     print(f'U: {U}, V: {V}, W: {W}')
    
    for i in range(len(inputs)):
        
#         print(f'U: {U.shape}, input: {inputs[i].shape}, v: {V.shape}, hidden: {hidden_state.shape}')
        hidden_state = tanh((np.dot(U, inputs[i]) + np.dot(V, hidden_states[-1])))
        output = np.dot(W, hidden_state)
#         print(f'hidden: {hidden_state>0}, output: {output}')
        
        hidden_states.append(hidden_state)
        outputs.append(output)
        
    return outputs, hidden_states

In [176]:
def clip_gradient_norm(grads, max_norm=0.25):
    """
    Prevents exploding gradient by clipping 
    Clips gradients to have max norm of max_norm
    """

    max_norm = float(max_norm)
    total_norm = 0

    # Using L2 norm squared
    for grad in grads:
        grad_norm = np.sum(np.power(grad, 2))
        total_norm += grad_norm

    total_norm = np.sqrt(total_norm)

    clip_coef = max_norm / (total_norm + 1e-6)

    if clip_coef < 1:
        for grad in grads:
            grad *= clip_coef

    return grads

In [268]:
def cross_entropy_loss(output, target):
    loss = 0
    for j in range(len(output)):
        
#         print(f'target: {target[j]}, out: {output[j]}, val: {output[j]}, log: {np.log(output[j] + 1e-9)}')
        loss += target[j] * np.log(output[j] + 1e-9) 
    
    return -loss    

In [219]:
def backward_pass(inputs, outputs, hidden_states, targets, params):
    U, V, W, bias_hidden, bias_output = params
    
    # Initialize gradients as zero
    d_U, d_V, d_W = np.zeros_like(U), np.zeros_like(V), np.zeros_like(W)
    d_bias_hidden, d_bias_output = np.zeros_like(bias_hidden), np.zeros_like(bias_output)
    
    d_hidden_next = np.zeros_like(hidden_states[0])
    loss = 0
    
    # Iterate backwards through elements
    for i in reversed(range(len(outputs))):
        
        # Calculate loss
#         print(f'{cross_entropy_loss(outputs[i], targets[i])}')
        loss += (cross_entropy_loss(softmax(outputs[i]), targets[i])/len(targets))
        
        # Backpropagate into output
        d_output = outputs[i].copy()
        d_output[np.argmax(targets[i])] -= 1
        
        # Backpropagate into W
#         print(f'h: {hidden_states[i].T.shape}, out: {d_output.shape}')
        d_W += np.dot(d_output, hidden_states[i].T)
        d_bias_output += d_output
        
        # Backpropagate into h
        d_h = np.dot(W.T, d_output) + d_hidden_next
        
        # Backpropagate through non-linearity (tanh)
        d_f = (1 - hidden_states[i]**2) * d_h
        d_bias_hidden += d_f
        
        # Backpropagate into U
#         print(f'h: {inputs[i].T.shape}, out: {d_f.shape}')
        d_U += np.dot(d_f, inputs[i].T)
        
        # Backpropagate into V
#         print(f'h: {hidden_states[i-1].T.shape}, out: {d_f.shape}')
        d_V += np.dot(hidden_states[i-1].T, d_f)
        d_hidden_next = np.dot(V.T, d_f)
        
    # Clip gradients
    grads = d_U, d_V, d_W, d_bias_hidden, d_bias_output
    grads = clip_gradient_norm(grads)
    
    return loss, grads
        
        

In [221]:
def optimizer(parameters, gradients, learning_rate=1e-3):
    for parameter, gradient in zip(parameters, gradients):
        parameter -= learning_rate * gradient
    
    return parameters

In [244]:
def encode_data(dataset, vocab_size):
    
    x, y = [], []
    for inputs, targets in dataset:
#         print(f'input: {len(inputs)}\ntargets{len(targets)}\n')
        x.append(one_hot_encode_sequence(inputs, vocab_size))
        y.append(one_hot_encode_sequence(targets, vocab_size))
        
#     print(f'lengths {len(x)}, {len(y)}')
    return (x, y)

In [275]:
def train(training_set, hidden_state, parameters, epochs=1000):
    
    training_loss = []
    inputs, targets = training_set
    for i in range(epochs):
        
        epoch_training_loss = 0
        for x, y in zip(inputs, targets):
            hidden_state = np.zeros_like(hidden_state)

            # Forward pass
            outputs, hidden_states = forward_pass(x, hidden_state, parameters)

            # Backward pass
            loss, gradients = backward_pass(x, outputs, hidden_states, y, parameters)
            if np.isnan(loss):
                raise ValueError('ERROR: Gradients have vanished')

            # Update parameters (optimizer)
            parameters = optimizer(parameters, gradients)
            epoch_training_loss += loss
            
        training_loss.append(epoch_training_loss/len(training_set))
    
        if i%100 == 0:
            print(f'Epoch {i}, training loss: {training_loss[-1]}')
        
    return parameters, training_loss

In [277]:
def validate(val_set, hidden_state, parameters, epochs=100):
    
    validation_loss = []
    inputs, targets = val_set
    for i in range(epochs):
        epoch_validation_loss = 0
        for x, y in zip(inputs, targets):
            hidden_state = np.zeros_like(hidden_state)
            
            #Forward pass
            outputs, hidden_states = forward_pass(x, hidden_state, parameters)
            
            # Backward pass
            loss, _ = backward_pass(x, outputs, hidden_states, y, parameters)
            if np.isnan(loss):
                raise ValueError('ERROR: Gradients have vanished')
            
        validation_loss.append(epoch_validation_loss/len(val_set))
        
        if i%100 == 0:
            print(f'Epoch {i}, validation loss: {validation_loss[-1]}')
            
    return validation_loss

In [291]:
def test(test_set, hidden_state, parameters, idx_to_word):
    inputs, targets = test_set
    results = defaultdict()
    for x in inputs:
        hidden_state = np.zeros_like(hidden_state)
        outputs, hidden_states = forward_pass(x, hidden_state, parameters)
        x_decoded = [ind_to_word[np.argmax(x[i])] for i in range(len(x))]
        y_decoded = [ind_to_word[np.argmax(output)] for output in outputs]
        x_decoded = ('').join(x_decoded)
        y_decoded = ('').join(y_decoded)
        results[x_decoded] = y_decoded
    return results
        

In [294]:
def rnn():
    
    # Constants
    epochs = 100
    hidden_size = 50
    hidden_state = np.zeros((hidden_size, 1))
    
    # Data Preparation
    sequences = generate_dataset()
    word_to_idx, idx_to_word, vocab_size = word_encoding(sequences)
    train_set, test_set, val_set = prepare_data(sequences)
    
    # Data encoding
    train_set = encode_data(train_set, vocab_size)
    test_set = encode_data(test_set, vocab_size)
    val_set = encode_data(val_set, vocab_size)
    
    # Initialize rnn
    parameters = init_rnn(hidden_size, vocab_size)
    training_loss, validation_loss = [], []
    
    # Train
    parameters, training_loss = train(train_set, hidden_state, parameters, epochs)
    
    # Validate
    validation_loss = validate(val_set, hidden_state, parameters, epochs)
    
    # Test
    results = test(test_set, hidden_state, parameters, idx_to_word)
    
    # Print results
    for key in results:
        print(f'Input: {key}, Output: {results[key]}')

In [295]:
rnn()

204, 25, 25
204, 25, 25
204, 25, 25
Epoch 0, training loss: [190.37475193]
Epoch 0, validation loss: 0.0
Input: aaaaaaaaabbbbbbbbbEO, Output: aaabbbbbbbbbbbbbbbbb
Input: aaabbbEO, Output: aaabbbbb
Input: aaaaaaaaaabbbbbbbbbbEO, Output: aaabbbbbbbbbbbbbbbbbbb
Input: aaaaaaaaaaabbbbbbbbbbbEO, Output: aaabbbbbbbbbbbbbbbbbbbbb
Input: aabbEO, Output: aabbbS
Input: abEO, Output: abbS
Input: aaaaabbbbbEO, Output: aaabbbbbbbbb
Input: aaaaaaaabbbbbbbbEO, Output: aaabbbbbbbbbbbbbbb
Input: aaaaaabbbbbbEO, Output: aaabbbbbbbbbbb
Input: aaaaaaabbbbbbbEO, Output: aaabbbbbbbbbbbbb


In [296]:
# LSTM